In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

In [2]:
# reading files of all btc_yuan and bitcoin:
bitcoin = pd.read_csv('../data/cleaned_data/bitcoin_clean.csv', index_col = 0)
yuan = pd.read_csv('../data/cleaned_data/yuan_clean.csv', index_col = 0)

In [3]:
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1049 entries, 0 to 1048
Data columns (total 2 columns):
Date     1049 non-null object
Index    1049 non-null float64
dtypes: float64(1), object(1)
memory usage: 24.6+ KB


In [4]:
yuan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837 entries, 0 to 836
Data columns (total 2 columns):
Date     837 non-null object
Value    837 non-null float64
dtypes: float64(1), object(1)
memory usage: 19.6+ KB


In [5]:
# creating a dataframe with all the values
btc_yuan = bitcoin.set_index('Date').join(yuan.set_index('Date'))

In [6]:
btc_yuan.head()

,Index,Value
Date,,
2017-01-01,998.33,NaN
2017-01-02,1021.75,NaN
2017-01-03,1043.84,6.96
2017-01-04,1154.73,6.93
2017-01-05,1013.38,6.88


In [7]:
btc_yuan.rename(columns={'Index': 'Bitcoin', 'Value':'Yuan'}, inplace = True)

In [8]:
# as I want to compare values on same days, I don't want NaN values, 
# but first I will check if there are many:
btc_yuan.isnull().sum()

Bitcoin      0
Yuan       222
dtype: int64

In [9]:
# The NaN values are due to non-opening sessions in the markets, so they are in bank 
# holidays or weekends. I will drop that rows as I can't compare the values
btc_yuan.drop(btc_yuan.query('Yuan == "NaN"').index, axis = 0, inplace = True)

In [10]:
btc_yuan.isnull().sum()

Bitcoin    0
Yuan       0
dtype: int64

In [11]:
btc_yuan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 827 entries, 2017-01-03 to 2019-11-15
Data columns (total 2 columns):
Bitcoin    827 non-null float64
Yuan       827 non-null float64
dtypes: float64(2)
memory usage: 19.4+ KB


In [12]:
btc_yuan.reset_index(inplace = True)

In [13]:
btc_yuan.head()

,Date,Bitcoin,Yuan
0,2017-01-03,1043.84,6.96
1,2017-01-04,1154.73,6.93
2,2017-01-05,1013.38,6.88
3,2017-01-06,902.20,6.92
4,2017-01-09,902.83,6.94


In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [19]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="Bitcoin Evolution vs Yuan-Dollar Evolution",
    font=dict(family="Arial, monospace",size=18, color="#000000"))

fig.add_trace(go.Scatter(x=btc_yuan['Date'], y=btc_yuan['Bitcoin'], name = 'Bitcoin'), secondary_y=True)
fig.add_trace(go.Scatter(x=btc_yuan['Date'], y=btc_yuan['Yuan'], name = 'Yuan-Dollar'))

fig.update_yaxes(title_text="Yuan-Dollar Exchange Rate")
fig.update_yaxes(title_text="Bitcoin Index", secondary_y=True)

fig.update_layout(legend_orientation="h")

fig.show()

In [16]:
import statsmodels.tsa.api as sm

In [17]:
# I will set the frequency to 44 days (2 months) to see the trend.
btc_trend = sm.seasonal_decompose(btc_yuan['Bitcoin'], freq = 44).trend
yuan_trend = sm.seasonal_decompose(btc_yuan['Yuan'], freq = 44).trend

In [20]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="Bitcoin Trend vs Yuan-Dollar Trend",
    font=dict(family="Arial, monospace",size=18, color="#000000"))

fig.add_trace(go.Scatter(x=btc_yuan['Date'], y=btc_trend, name = 'Bitcoin'), secondary_y=True)
fig.add_trace(go.Scatter(x=btc_yuan['Date'], y=yuan_trend, name = 'Yuan_Dollar'))

fig.update_yaxes(title_text="Yuan-Dollar Exchange Rate")
fig.update_yaxes(title_text="Bitcoin Index", secondary_y=True)

fig.update_layout(legend_orientation="h")

fig.show()